# Save & Load

Based on **Patric Loeber** video: https://www.youtube.com/watch?v=LwM7xabuiIc&t=6078s

In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# normalize
x_train, x_test = x_train / 255.00, x_test / 255.00

In [6]:
# model from tf_03
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

In [7]:
# config
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim = keras.optimizers.legacy.Adam(learning_rate=0.001)
metrics = [keras.metrics.SparseCategoricalAccuracy()]


In [8]:
# compile
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [9]:
# fit / training
model.fit(x_train, y_train, batch_size=64, epochs=5, shuffle=True, verbose=2)

print('Evaluate')
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
938/938 - 1s - loss: 0.3002 - sparse_categorical_accuracy: 0.9157 - 601ms/epoch - 640us/step
Epoch 2/5
938/938 - 1s - loss: 0.1387 - sparse_categorical_accuracy: 0.9594 - 502ms/epoch - 535us/step
Epoch 3/5
938/938 - 0s - loss: 0.0981 - sparse_categorical_accuracy: 0.9709 - 475ms/epoch - 506us/step
Epoch 4/5
938/938 - 0s - loss: 0.0752 - sparse_categorical_accuracy: 0.9774 - 472ms/epoch - 503us/step
Epoch 5/5
938/938 - 0s - loss: 0.0589 - sparse_categorical_accuracy: 0.9824 - 474ms/epoch - 506us/step
Evaluate
313/313 - 0s - loss: 0.0785 - sparse_categorical_accuracy: 0.9765 - 141ms/epoch - 449us/step


[0.07849851250648499, 0.9764999747276306]

## 1) Save whole model

We can use SaveModel format or HDF5 format. Using one or another depends on the ending that we use.

In [10]:
model.save("nn") # creates whole folder with files
# if we specify the ending tf will save it like HDF5
model.save("nn.h5")

INFO:tensorflow:Assets written to: nn/assets


INFO:tensorflow:Assets written to: nn/assets
/Users/florian/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Both of methods works fine but in tf documentation it is recommended to use SaveModel format

## Loading model

In [11]:
new_model = keras.models.load_model("nn.h5")
new_model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0785 - sparse_categorical_accuracy: 0.9765 - 128ms/epoch - 408us/step


[0.07849851250648499, 0.9764999747276306]

## 2) Save only weights, here we also have same two options

In [12]:
model.save_weights("nn_weights.h5")

In [13]:
# initialize
model.load_weights("nn_weights.h5")

## 3) Save only architecture, to_json

In [14]:
json_string = model.to_json()

with open("nn_weights.json", "w") as f:
    f.write(json_string)

In [15]:
# load architecture
with open("nn_weights.json") as f:
    loaded_json_string = f.read()

In [16]:
new_model = keras.models.model_from_json(loaded_json_string)
print(new_model.summary()) # We still have to train it because it does not have weights

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
